In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras, Tensor
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import matplotlib.image as img
import math
import scipy
import scipy.signal


In [ ]:
#mounts to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_model():
  new_model = tf.keras.models.load_model('/content/drive/My Drive/VIP - Plant Recognition/Collab Notebooks/cnn_model.h5')
  return new_model

def greyscale(imageArray:np.ndarray):
    # initialize 3D array representation of Image
    # initialize numpy array of zeros for output
    arr = np.zeros((imageArray.shape[0],imageArray.shape[1]))
    # Sum of RGB values according to BT.709 at each pixel
    # Y = 0.2126R + 0.7152G + 0.0722B
    
    arr = imageArray[:,:,0] * 0.2126 + imageArray[:,:,1] *  0.7152 + imageArray[:,:,2] * 0.0722
    return arr

def resizeImage(oldImageArr: np.array, new_width, new_height):    
    #image.shape returns (height, width)
    width = oldImageArr.shape[1]
    height = oldImageArr.shape[0]
    
    #initialize numpy array for new image
    new_image = np.ndarray((new_height, new_width)) 
    
    #Iterates through each pixel in the new image
    #Chooses the pixel value based by approximating 
    #the corresponding pixel location in the old image array
    #The pixel at the approximated location is used as the new
    #pixel value
    for x in range(new_width):
        for y in range(new_height):
            new_image[y][x] = oldImageArr[int(y*(height/new_height)) , int(x*(width/new_width))] 
            
    return new_image

#Bruna's function
''' This is the final version of the filter, which uses both the x filter and the y filter '''
def both_filters_greyscale1(image):

    #kernel to traverse x
    horizontal_filter = [[-1,0,1],
                        [-2,0,2],
                        [-1,0,1]]

    #kernel to traverse y
    vertical_filter =   [[-1,-2,-1],
                        [0,0,0],
                        [1,2,1]]

    height, width = image.shape
    
    new_image = np.zeros_like(image)
    
    for i in range(1, height - 2):
        for j in range(1, width-2):
            local_pixels = image[i-1:i+2, j-1:j+2]
            
            horizontal_transformed_pixels = horizontal_filter * local_pixels
            horizontal_score = (horizontal_transformed_pixels.sum())    #these values are not plot to 0-1 because otherwise 
                                                                        #the result becomes too dim.
        
            vertical_transformed_pixels = vertical_filter * local_pixels 
            vertical_score = (vertical_transformed_pixels.sum())
            
            edge_score = (vertical_score**2 + horizontal_score**2)**0.5
            new_image[i][j] = edge_score 
    
    return new_image

# 9/18/2022 (ah)
def gaussianBlur(imageArr: np.array, radius: int):
    sigma = 2#max(float(radius/2), 1)
    # Computes Kernal size from radius
    # EX: radius = 1 -> kernalSize = 3
    # 3x3 Kernal
    kernalSize = (2 * radius) + 1
    outputArr = np.zeros(imageArr.shape)
    #Initialize kernal
    kernal = np.zeros((kernalSize, kernalSize))
    sum = 0
    
    # Iterate through each element in kernal to determine value
    # Uses 2d gaussian function, center position is 0
    # x and y range from -radius to radius 
    # 'value' is the output from gaussian function at point (x,y)
    for x in range(-radius, radius+1):
        for y in range(-radius, radius+1):
            expNum = float(-(x*x + y*y))
            expDenom = (2 * sigma * sigma)
            
            expResult = math.exp(expNum / expDenom)
            value = (expResult / (2 * math.pi * sigma * sigma))
            kernal[x + radius][y + radius] = value
            sum += value
    
    #Normalized the kernal
    for x in range(kernalSize):
        for y in range(kernalSize):
            kernal[x][y] /= sum
    
    #TODO: blur edge pixels
    #This is where the new value for each pixel is calculated
    #Iterates over every pixel that allows for operation
    outputArr = scipy.signal.convolve(kernal, imageArr)
    
    
    return outputArr

#NOTE: below is a function that I wrote to try to see if I could decrease the size of the shapes in the background. I don't think this will be useful.
def nonBinaryThreshold(image, threshValue1, threshValue2):#image is in greyscale
    output = np.zeros_like(image)
    rows, columns =  output.shape

    for row in range(0, rows-1):
        for col in range(0, columns-1):
            if image[row][col] >= threshValue1:
                output[row][col] = 255
            elif image[row][col] < threshValue1 and image[row][col] >= threshValue2:
                output[row][col] = 127
            else:
                output[row][col] = 0

    return output


In [ ]:
# filePath = "/content/drive/MyDrive/test_images/mmh/mmh39.jpg"
filePath = "/content/drive/MyDrive/Flower Images/Diablo Cosmo/dc14.jpg"
img_width = 128
img_height = 128

image = img.imread(filePath)
grey = greyscale(image)
blur = gaussianBlur(grey, 5)
sobel = both_filters_greyscale1(blur)
thres = nonBinaryThreshold(sobel, 150, 50)
resize_image = resizeImage(thres, img_width, img_height)
tensor_image = tf.convert_to_tensor(resize_image)
tensor_image = tf.expand_dims(tensor_image, axis=0)
tensor_image = tf.reshape(tensor_image, [128,128,1])
tensor_image = tf.expand_dims(tensor_image, axis=0)


#/content/drive/MyDrive/test_images/mmh39.jpg
# plt.imshow(tensor_image, cmap='gray')
# plt.show()


FileNotFoundError: ignored

In [ ]:
model = get_model()
prediction_list = model.predict(tensor_image, verbose=0)[0]
prediction = max(prediction_list)
prediction_index = list(prediction_list).index(prediction)
classes = ['diablo', 'mmh', 'vis']
ans = classes[prediction_index]
print(ans)
print(prediction_list)


mmh
[0.30018508 0.49658036 0.20323454]


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  "/content/drive/MyDrive/test_images",
  label_mode='int',
  color_mode="grayscale",
  shuffle="False",
  seed=123,
  image_size=(128, 128),
  batch_size=1)

# model = get_model()
# for images, labels in val_ds.take(1):
#   print(images.shape)
#   prediction = model.predict(images)

# #['diablo', 'mmh', 'vis']
# fig, ax = plt.subplots(6)
# fig.set_size_inches(12,12)
# print(labels)
# for idx in range(6):
#     print(labels[idx])
#     [print(x) for x in prediction[idx]]
#     image = tf.transpose(images[idx], perm=[2,0,1])
#     ax[idx].imshow(image[0], cmap="gray")

Found 6 files belonging to 3 classes.
